In [1]:
import nbimporter
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch import nn
from torch.optim import SGD
from sklearn.metrics import accuracy_score
from os.path import join
import torch
import numpy as np
from PIL import Image
from os import path

import python_file.dataclass as StreetSign
import python_file.function as Function
import python_file.network as Network

import matplotlib.pyplot as plt

np.random.seed(1328)
torch.random.manual_seed(1328);

In [2]:
#resize normale
tran = transforms.Compose([StreetSign.Rescale(32),StreetSign.RandomCrop(32),StreetSign.ToTensor()])

In [3]:
datatrain = StreetSign.StreetSignDataset('DITS-full\DITS-full\DITS-detection\class\classes.csv','DITS-full\DITS-full\DITS-detection\class\image',tran)
datatest = StreetSign.StreetSignDataset('DITS-full\DITS-full\DITS-detection\detection_test\day\classe.csv','DITS-full\DITS-full\DITS-detection\detection_test\day',tran)

In [4]:
datatrain_loader = DataLoader(datatrain, batch_size=1024, shuffle=True)
#datatrain_loader = DataLoader(cifar100_test, batch_size=1024, num_workers=2)
datatest_loader = DataLoader(datatest, batch_size=1024, shuffle=True)
#datatest_loader = DataLoader(cifar100_test, batch_size=1024, num_workers=2)

In [5]:
#number of epoch
epoche = 5

In [6]:
lenet_datatrain = Network.LeNetColor()

lenet_datatrain = Function.train_classifier(lenet_datatrain, datatrain_loader, datatest_loader, \
                                            'lenet_dataset', epochs=2)

lenet_data_test_predictions, data_labels_test = Function.test_classifier(lenet_datatrain,
                                                                         datatest_loader)
print("Accuracy LeNetColor su StreetSign: %0.2f" % \
      accuracy_score(data_labels_test,lenet_data_test_predictions))

RuntimeError: stack expects each tensor to be equal size, but got [1] at entry 0 and [3] at entry 25

In [7]:
minialexnet_datatrain = Network.MiniAlexNet()

minialexnet_datatrain = Function.train_classifier(minialexnet_datatrain, datatrain_loader, datatest_loader, \
                                                  'minialexnet_dataset', epochs=2)

minialexnet_data_test_predictions, data_labels_test = Function.test_classifier(minialexnet_datatrain,
                                                                               datatest_loader)
print("Accuracy MiniAlexNet su StreetSign: %0.2f" % \
      accuracy_score(data_labels_test,minialexnet_data_test_predictions))

RuntimeError: stack expects each tensor to be equal size, but got [1] at entry 0 and [2] at entry 119

In [8]:
minialexnetV2_datatrain = Network.MiniAlexNetV2()

minialexnetV2_datatrain = Function.train_classifier(minialexnetV2_datatrain, datatrain_loader, datatest_loader, \
                                                    'minialexnetV2_dataset', epochs=2)

minialexnetV2_data_test_predictions, data_labels_test = Function.test_classifier(minialexnetV2_datatrain,
                                                                               datatest_loader)
print("Accuracy MiniAlexNetV2 su StreetSign: %0.2f" % \
      accuracy_score(data_labels_test,minialexnetV2_data_test_predictions))

RuntimeError: stack expects each tensor to be equal size, but got [1] at entry 0 and [2] at entry 227